In [2]:
from difflib import  unified_diff as diff
import sqlite3
from requests import Session
#from xml.etree import ElementTree as XML
from datetime import datetime
from time import sleep
#conn = None
if 'conn' in dir():
  print('Previous Database Closed')
  conn.close()
conn = sqlite3.connect('database.db')
cur = conn.cursor()


The Features we'll be saving as follows:
- Title of the page
- comment of the edit
- revision ID of the current and previous edit
- difference of the content
- Username or IP

Because these can be hiiden by an admin.

In [3]:
SQL_INIT = """
CREATE TABLE IF NOT EXISTS `Revisions` (
  `RevID` INTEGER PRIMARY KEY,
  `Title` TEXT NOT NULL,
  `ParentID` INTEGER NULL DEFAULT NULL,
  `Comment` TEXT NULL DEFAULT NULL,
  `Anonymous` BOOLEAN DEFAULT FALSE,
  `Editor` VARCHAR(90) NULL DEFAULT NULL,
  `Text` TEXT NULL DEFAULT NULL,
  `Deleted` BOOLEAN DEFAILT FALSE
) WITHOUT ROWID;
CREATE INDEX IF NOT EXISTS `ParentID` ON `Revisions`(`ParentID`);
CREATE TABLE IF NOT EXISTS `Labels` (
  `RevID` INTEGER REFERENCES `Revisions` (`RevID`),
  `Labeller` VARCHAR(100) NOT NULL,
  `Vandalism` BOOLEAN DEFAULT FALSE,
  `Timestamp` TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""
SQL_INSERT_REVISION = """
INSERT INTO `Revisions`
  (`Title` ,`RevID`, `ParentID`, `Comment`, `Anonymous`, `Editor`, `Deleted`, `Text`)
VALUES
  (?, ?,?,?,?,?,?, ?);
"""
SQL_UPDATE_REVISION = """
UPDATE `Revisions` SET `Text` = ? WHERE `ParentID` = ?
"""
cur.executescript(SQL_INIT)

In [13]:
# the parameter to pass on
mw = 'http://www.mediawiki.org/xml/export-0.11/' #The namespace
api_endpoint = 'https://bn.wikipedia.org/w/api.php' #Wikipedia API endpoint
ses = Session()
cache={}
insert_list = set()
backlog = set() # The Parent IDs which should be fetched
revision_backlog = set()
last_time = datetime.fromisoformat('2021-08-28T07:34:00')
params_live = {
	"action": "query",
	"format": "json",
	"prop": "revisions",
	"utf8": 1,
	"indexpageids" : 1,
	"rvprop": "comment|content|ids|flags|user|timestamp",
	"rvslots": "main"
}
total = 0
suceed = set()
success = 0

In [14]:
def process_batch(query={}, ids=[], cache={}):
  """This function will be called on update statement each batch of results
  `queries`: The list of the values
  `cache`: A dictionary with `ParentID` as key and `Current revision` as Value
  """
  global backlog, revision_backlog
  for id in ids:
    page = query[id]
    print(page['title'])
    if 'revisions' not in page:
      revision_backlog.add(page['title'])
      print('\tRevision not found')
      continue
    if page['title'] in revision_backlog:
      print('\t\t', page['title'], 'Revived')
      revision_backlog.remove(page['title'])
    title = page['title']
    revision = page['revisions'][0]
    current_text = revision['slots']['main']['*'].splitlines()
    current_id = revision['revid']
    parent_id = revision['parentid']
    comment = revision['comment']
    editor = revision['user']
    insertable_text = None
    
    backlog.discard(current_id)
    print(f'\tBacklog size : {len(backlog)}')
    if current_id in cache:
      print("""\tPut the difference""")
      insertable_text = '\n'.join(diff(cache[current_id], current_text))
      del cache[current_id]
      # if parent_id:
      #   # Add parent to the backlog
      #   backlog.add(parent_id)'
      #   print('Push to cache')
      #   cache[parent_id] = current_text
      print('\tUpdate parameters')
      yield insertable_text, current_id
    else:
      print(f'\tcache[{current_id}] not found')
      if not parent_id:
        print('\tFirst Revision', title)
        """
        ParentID does not exist
        So this is the first revision
        Insert it in raw format
        """
        parent_id = None
        insertable_text = '\n'.join(diff('', current_text))
      else:
        cache[parent_id] = current_text
        backlog.add(parent_id)
      #Insert into database
      insert_list.add(
          (
          title,
          current_id, parent_id,
          comment, 1, editor, 1,
          insertable_text)
      )

In [17]:
def fetch(ids = [], last_time = 0, grccontinue=None, rvcontinue=None, limit=50, idlimit=50):
  global cache, insert_list, backlog
  k=5
  cont=None
  while k:
    print('Epoch', k)
    if cont:
      print('\t\tContinue---------------------',success, total)
      # Revision continual
      data = {**params_live,
        "generator": "recentchanges",
        "grcstart": last_time,
        "grcdir": "newer",
        "grclimit": limit,
        "grctoponly": 1,
        'continue':cont,
        'grctag' : 'mw-rollback'
      }
      if grccontinue:
        data['grccontinue'] = grccontinue
      if rvcontinue:
        data['rvcontinue']= rvcontinue
    elif ids:
      ids = list(map(str, backlog.union(ids)))
      next_ids = []
      if len(ids) > idlimit:
        next_ids, ids = ids[idlimit:], ids[:idlimit]
      data = {**params_live, 'revids':'|'.join(ids)}
      ids = next_ids
    elif last_time:
      print('Fetching Live Changes')
      if isinstance(last_time, str):
        last_time = datetime.fromisoformat(last_time)
      data = {**params_live,
        "generator": "recentchanges",
        "grcstart": last_time.isoformat(timespec='seconds'),
        "grcdir": "newer",
        "grclimit": limit,
        "grctoponly": 1,
        'grctag' : 'mw-rollback'
      }
    else:
      print('neither Last Time nor Revision ID defined')
      
    res = ses.post(api_endpoint, params=data, headers={'content-type':'application/json'}).json()
    if 'query' not in res:
      print('Query Not found')
      print(res)
      return
    print(data)
    cur.executemany(SQL_UPDATE_REVISION, process_batch(
      ids=res['query']['pageids'],
      query=res['query']['pages'],
      cache=cache,
      last_time=last_time
      ))
    for i in insert_list:
      try:
        cur.execute(SQL_INSERT_REVISION, i)
      except sqlite3.IntegrityError as e:
        print(f'Error {i[0]}')
        backlog.discard(i[1])
        pass
    insert_list = set()
    if 'continue' in res:
      cont = res['continue']['continue']
      print('Continual', res['continue'])
      if 'grccontinue' in res['continue']:
        grccontinue = res['continue']['grccontinue']
      else:
        grccontinue = None
      if 'rvcontinue' in res['continue']:
        rvcontinue = res['continue']['rvcontinue']
      else:
        rvcontinue = None
    else:
      cont = None
    if backlog:
      ids = backlog
    k -= 1
    
    print(f'Success Rate : {len(suceed)} / {total}  = {len(suceed) * 100/ total : 0.2f}%')
    print(' \tSleeping ------------------------------------')
    sleep(2)
  
#cache={}
# fetch(ids=[123456, 123457]))
fetch(last_time=last_time)

Epoch 5
Fetching Live Changes
{'action': 'query', 'format': 'json', 'prop': 'revisions', 'utf8': 1, 'indexpageids': 1, 'rvprop': 'comment|content|ids|flags|user|timestamp', 'rvslots': 'main', 'generator': 'recentchanges', 'grcstart': '2021-08-28T07:34:00', 'grcdir': 'newer', 'grclimit': 50, 'grctoponly': 1, 'grctag': 'mw-rollback'}
আমর ইবনুল আস মসজিদ
	Backlog size : 14
	cache[5370095] not found
উত্তরপ্রদেশ
	Backlog size : 14
	cache[5370547] not found
এডুয়ার্ডো স্যাভেরিন
	Backlog size : 14
	cache[5369895] not found
এলিটা করিম
	Backlog size : 14
	cache[5370431] not found
জাতীয় গণতান্ত্রিক একতা
	Backlog size : 14
	cache[5370574] not found
জিন সিমন্স
	Backlog size : 14
	cache[5369881] not found
টুইটার
	Backlog size : 14
	cache[5369843] not found
তা’মীরুল মিল্লাত কামিল মাদরাসা
	Backlog size : 14
	cache[5370433] not found
পিপীলিকা (অনুসন্ধান ইঞ্জিন)
	Backlog size : 14
	cache[5369844] not found
প্রধান পরিচালন কর্মকর্তা
	Backlog size : 14
	cache[5370773] not found
মোহাম্মদ শাকিল (ক্রিকেটার)


In [18]:
#cur.execute('DELETE FROM Revisions')
for i in cur.execute('SELECT * FROM Revisions'):
    print(i[6])

--- 

+++ 

@@ -34,7 +34,7 @@

 
 টুইট লেখার জন্য সদস্যরা সরাসরি টুইটার ওয়েবসাইট ব্যবহার করতে পারেন। এছাড়াও, মোবাইল ফোন বা এসএমএসের মাধ্যমেও টুইট লেখার সুযোগ রয়েছে।<ref>{{ওয়েব উদ্ধৃতি |ইউআরএল=http://help.twitter.com/entries/14226-how-to-find-your-twitter-short-long-code |শিরোনাম=Using Twitter With Your Phone |উক্তি=We currently support 2-way (sending and receiving) Twitter SMS via short codes and 1-way (sending only) via long codes. |প্রকাশক=Twitter Support |সংগ্রহের-তারিখ=2010-06-01 |ভাষা=ইংরেজি |আর্কাইভের-ইউআরএল=https://web.archive.org/web/20100315200218/http://help.twitter.com/entries/14226-how-to-find-your-twitter-short-long-code |আর্কাইভের-তারিখ=২০১০-০৩-১৫ |অকার্যকর-ইউআরএল=হ্যাঁ }}</ref> টুইটারের মূল কার্যালয় [[মার্কিন যুক্তরাষ্ট্র|মার্কিন যুক্তরাষ্ট্রের]] [[সান ফ্রান্সিস্কো]] শহরে। এছাড়াও, [[টেক্সাস|টেক্সাসের]] সান অ্যান্টোনিও এবং [[ম্যাসাচুসেটস্|ম্যাসাচুসেটসের]] [[বস্টন|বস্টনে]] টুইটারের [[সার্ভার]] ও শাখা কার্যালয় রয়েছে।
 
-২০০৬ সালের মার্চ মাসে টুইটারের যাত্রা শুরু হয়